<a href="https://colab.research.google.com/github/Dipendra-Pal/AI-Lab1-Dipendra-Pal/blob/main/UtilityBasedAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random
from collections import deque

class VacuumUtilityAgent:
    def __init__(self, size=(5, 5)):
        self.size = size
        self.grid = [[random.choice(['C', 'D']) for _ in range(size[1])] for _ in range(size[0])]
        self.pos = [0, 0]
        self.memory = {(0, 0)}  # Track visited positions
        self.cleaned = 0
        self.total_dirty = sum(row.count('D') for row in self.grid)

    def done(self):
        return self.cleaned >= self.total_dirty

    def get_utility(self, pos, action):
        """Calculate utility for a given position and action."""
        x, y = pos
        if action == 'clean' and self.grid[x][y] == 'D':
            return 10  # High utility for cleaning dirty cell
        if action == 'move':
            new_x, new_y = pos
            if (new_x, new_y) in self.memory:
                return 1  # Low utility for revisiting
            # Estimate distance to nearest dirty cell
            distance = self.distance_to_dirty((new_x, new_y))
            if distance == 0:
                return 8  # High utility if dirty cell is at position
            return 5 / (distance + 1)  # Higher utility for closer dirty cells
        return 0

    def distance_to_dirty(self, start):
        """Find shortest distance to a dirty cell using BFS."""
        queue = deque([(start, 0)])
        visited = {start}
        while queue:
            (x, y), dist = queue.popleft()
            if self.grid[x][y] == 'D' and (x, y) not in self.memory:
                return dist
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < self.size[0] and 0 <= ny < self.size[1] and (nx, ny) not in visited:
                    queue.append(((nx, ny), dist + 1))
                    visited.add((nx, ny))
        return float('inf')  # No dirty cells found

    def act(self):
        """Choose action with highest utility."""
        x, y = self.pos
        actions = []

        # Consider cleaning current cell
        if self.grid[x][y] == 'D':
            actions.append(('clean', (x, y), self.get_utility((x, y), 'clean')))

        # Consider moving to adjacent cells
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size[0] and 0 <= ny < self.size[1]:
                actions.append(('move', (nx, ny), self.get_utility((nx, ny), 'move')))

        if not actions:
            return "Done"

        # Choose action with highest utility
        action, next_pos, _ = max(actions, key=lambda x: x[2])

        if action == 'clean':
            self.grid[x][y] = 'C'
            self.cleaned += 1
            self.memory.add((x, y))
            return f"Cleaned ({x},{y})"

        self.pos = list(next_pos)
        self.memory.add(next_pos)
        return f"Moved to ({next_pos[0]},{next_pos[1]})"

    def display(self):
        """Display grid with agent position."""
        for i in range(self.size[0]):
            row = ['V' if [i, j] == self.pos else self.grid[i][j] for j in range(self.size[1])]
            print(' '.join(row))
        print()

def main():
    agent = VacuumUtilityAgent()
    agent.display()
    for step in range(100):
        if agent.done():
            print("All clean!")
            break
        print(f"Step {step + 1}: {agent.act()}")
        agent.display()

if __name__ == "__main__":
    main()

V D D D C
D C C C D
C D D D C
C D D C D
D D C D C

Step 1: Moved to (1,0)
C D D D C
V C C C D
C D D D C
C D D C D
D D C D C

Step 2: Cleaned (1,0)
C D D D C
V C C C D
C D D D C
C D D C D
D D C D C

Step 3: Moved to (2,0)
C D D D C
C C C C D
V D D D C
C D D C D
D D C D C

Step 4: Moved to (2,1)
C D D D C
C C C C D
C V D D C
C D D C D
D D C D C

Step 5: Cleaned (2,1)
C D D D C
C C C C D
C V D D C
C D D C D
D D C D C

Step 6: Moved to (3,1)
C D D D C
C C C C D
C C D D C
C V D C D
D D C D C

Step 7: Cleaned (3,1)
C D D D C
C C C C D
C C D D C
C V D C D
D D C D C

Step 8: Moved to (4,1)
C D D D C
C C C C D
C C D D C
C C D C D
D V C D C

Step 9: Cleaned (4,1)
C D D D C
C C C C D
C C D D C
C C D C D
D V C D C

Step 10: Moved to (4,0)
C D D D C
C C C C D
C C D D C
C C D C D
V C C D C

Step 11: Cleaned (4,0)
C D D D C
C C C C D
C C D D C
C C D C D
V C C D C

Step 12: Moved to (3,0)
C D D D C
C C C C D
C C D D C
V C D C D
C C C D C

Step 13: Moved to (2,0)
C D D D C
C C C C D
V C D D C
C C D C D